In [1]:
import numpy as np 
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler, normalize

In [2]:
pwd

'/home/zaids_device/Downloads/TPS may'

In [6]:
train = pd.read_csv('/home/zaids_device/Downloads/TPS may/tabular-playground-series-may-2022/train.csv')
test = pd.read_csv('/home/zaids_device/Downloads/TPS may/tabular-playground-series-may-2022/test.csv')
submission = pd.read_csv('/home/zaids_device/Downloads/TPS may/tabular-playground-series-may-2022/sample_submission.csv')

In [7]:
#Reduce Memory Usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)  

            
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [8]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 50.64 Mb (77.7% reduction)
Mem. usage decreased to 38.72 Mb (77.3% reduction)


In [9]:
def obj_preprocessing(df, cols):
            
    def start_end(df, cols):
        for col in cols:
            df[col+"_s_e"]=df[col].apply(lambda x: x[0]+x[-1])
        return df
    
    df = start_end(df, cols)
    print("Done")
    return df
#train = obj_preprocessing(train, ["f_27"])
#test = obj_preprocessing(test, ["f_27"])

In [10]:
#train.drop(["f_27_s_e","id"],1,inplace=True)
#test.drop(["f_27_s_e","id"],1,inplace=True)

In [11]:
train

,id,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,...,f_22,f_23,f_24,f_25,f_26,f_27,f_28,f_29,f_30,target
0,0,-1.373047,0.238892,-0.243408,0.567383,-0.647949,0.839355,0.113159,1,5,...,-2.541016,0.767090,-2.730469,-0.208130,1.363281,ABABDADBAB,67.6250,0,0,0
1,1,1.697266,-1.709961,-2.230469,-0.545898,1.113281,-1.551758,0.447754,1,3,...,2.277344,-0.633789,-1.216797,-3.781250,-0.058319,ACACCADCEB,377.0000,0,0,1
2,2,1.681641,0.616699,-1.027344,0.810547,-0.608887,0.113953,-0.708496,1,0,...,-1.385742,-0.520508,-0.009125,2.789062,-3.703125,AAAEABCKAD,-195.6250,0,2,1
3,3,-0.118164,-0.587891,-0.804688,2.085938,0.371094,-0.128784,-0.282471,3,2,...,0.572754,-1.653320,1.685547,-2.533203,-0.608398,BDBBAACBCB,210.8750,0,0,1
4,4,1.148438,-0.176514,-0.665039,-1.101562,0.467773,0.500000,0.407471,3,3,...,-3.912109,-1.430664,2.126953,-3.306641,4.371094,BDBCBBCHFE,-217.2500,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,899995,1.379883,-0.038879,0.597168,0.854492,0.684082,-1.058594,1.310547,2,1,...,-1.594727,0.521973,0.833008,2.714844,1.290039,BABBCBBBED,455.0000,0,2,1
899996,899996,-1.370117,0.044830,0.015457,0.376465,-0.380615,-0.831055,-1.798828,4,1,...,2.414062,-0.674805,-0.412109,-0.030441,-3.144531,BBBGBBDQBE,134.7500,0,1,0
899997,899997,1.385742,-0.960938,0.726074,-0.132812,0.874023,-0.245361,-1.045898,0,0,...,-0.151978,-4.562500,-1.249023,1.793945,2.253906,AEBEDBBHBA,-99.5625,0,1,0
899998,899998,-1.590820,-0.509766,-1.715820,-0.250000,1.360352,1.650391,-0.058594,0,2,...,2.423828,2.109375,0.561035,-2.150391,1.019531,ADBAAADDAE,47.8125,1,2,0


In [12]:
for i in range(10):
    train["f_27_"+str(i+1)] = train["f_27"].apply(lambda x: x[i])
    test["f_27_"+str(i+1)] = test["f_27"].apply(lambda x: x[i])

In [13]:
le = LabelEncoder()
labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
le.fit(labels)

LabelEncoder()

In [14]:
for i in range(10):
    train["f_27_"+str(i+1)] = le.transform(train["f_27_"+str(i+1)])
    test["f_27_"+str(i+1)] = le.transform(test["f_27_"+str(i+1)])
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
train.drop(["f_27", "id"], axis=1, inplace=True)
test.drop(["f_27","id"], axis=1, inplace=True)

In [15]:
y = train.target
train.drop("target",1,inplace=True)
train=train.join(y)

/tmp/ipykernel_34906/1189427944.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train.drop("target",1,inplace=True)


In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900000 entries, 0 to 899999
Data columns (total 41 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   f_00     900000 non-null  float16
 1   f_01     900000 non-null  float16
 2   f_02     900000 non-null  float16
 3   f_03     900000 non-null  float16
 4   f_04     900000 non-null  float16
 5   f_05     900000 non-null  float16
 6   f_06     900000 non-null  float16
 7   f_07     900000 non-null  int8   
 8   f_08     900000 non-null  int8   
 9   f_09     900000 non-null  int8   
 10  f_10     900000 non-null  int8   
 11  f_11     900000 non-null  int8   
 12  f_12     900000 non-null  int8   
 13  f_13     900000 non-null  int8   
 14  f_14     900000 non-null  int8   
 15  f_15     900000 non-null  int8   
 16  f_16     900000 non-null  int8   
 17  f_17     900000 non-null  int8   
 18  f_18     900000 non-null  int8   
 19  f_19     900000 non-null  float16
 20  f_20     900000 non-null  

In [17]:
train

,f_00,f_01,f_02,f_03,f_04,f_05,f_06,f_07,f_08,f_09,...,f_27_2,f_27_3,f_27_4,f_27_5,f_27_6,f_27_7,f_27_8,f_27_9,f_27_10,target
0,-1.373047,0.238892,-0.243408,0.567383,-0.647949,0.839355,0.113159,1,5,1,...,1,0,1,3,0,3,1,0,1,0
1,1.697266,-1.709961,-2.230469,-0.545898,1.113281,-1.551758,0.447754,1,3,4,...,2,0,2,2,0,3,2,4,1,1
2,1.681641,0.616699,-1.027344,0.810547,-0.608887,0.113953,-0.708496,1,0,2,...,0,0,4,0,1,2,10,0,3,1
3,-0.118164,-0.587891,-0.804688,2.085938,0.371094,-0.128784,-0.282471,3,2,1,...,3,1,1,0,0,2,1,2,1,1
4,1.148438,-0.176514,-0.665039,-1.101562,0.467773,0.500000,0.407471,3,3,0,...,3,1,2,1,1,2,7,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,1.379883,-0.038879,0.597168,0.854492,0.684082,-1.058594,1.310547,2,1,2,...,0,1,1,2,1,1,1,4,3,1
899996,-1.370117,0.044830,0.015457,0.376465,-0.380615,-0.831055,-1.798828,4,1,2,...,1,1,6,1,1,3,16,1,4,0
899997,1.385742,-0.960938,0.726074,-0.132812,0.874023,-0.245361,-1.045898,0,0,6,...,4,1,4,3,1,1,7,1,0,0
899998,-1.590820,-0.509766,-1.715820,-0.250000,1.360352,1.650391,-0.058594,0,2,2,...,3,1,0,0,0,3,3,0,4,0


In [18]:
std = StandardScaler()
train_nx = std.fit_transform(train.drop("target", 1))
test_nn = std.transform(test)

/tmp/ipykernel_34906/4081180835.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  train_nx = std.fit_transform(train.drop("target", 1))


In [19]:
train_nn = pd.DataFrame(train_nx).join(train.target)

In [20]:
train_np = np.array(train_nn)
test_np = np.array(test_nn)

In [21]:
test_np

array([[ 0.44340651,  0.17341362, -1.00065979, ...,  0.28561839,
        -1.26442002, -0.17164158],
       [-0.60585676, -0.30707696,  0.62594539, ..., -0.59079541,
        -0.01435285, -0.77677095],
       [ 0.30458016,  2.44612308,  0.2451597 , ...,  0.11033563,
         1.23571431,  1.03861715],
       ...,
       [-0.33187024, -0.33028907, -1.18611133, ...,  1.16203218,
        -0.63938644, -1.38190031],
       [-2.03322633, -1.24044794,  0.96317436, ...,  0.63618391,
         0.61068073,  1.03861715],
       [-0.08568566, -0.00329044,  2.22424481, ..., -0.41551265,
         1.8607479 ,  1.64374651]])

In [22]:
train_nn = torch.from_numpy(train_np)
test_nn = torch.from_numpy(test_np)

In [23]:
train_nn

tensor([[-1.3743,  0.2379, -0.2445,  ..., -1.2644, -0.7768,  0.0000],
        [ 1.6994, -1.7125, -2.2305,  ...,  1.2357, -0.7768,  1.0000],
        [ 1.6838,  0.6160, -1.0280,  ..., -1.2644,  0.4335,  1.0000],
        ...,
        [ 1.3876, -0.9629,  0.7245,  ..., -0.6394, -1.3819,  0.0000],
        [-1.5923, -0.5113, -1.7161,  ..., -1.2644,  1.0386,  0.0000],
        [-0.6367, -0.4275, -1.8274,  ..., -0.0144,  1.0386,  1.0000]],
       dtype=torch.float64)

In [24]:
def transfer_device(data, device="cuda"):
    if torch.cuda.is_available():
        data = data.to('cuda')
    return data

In [25]:
train_nn=transfer_device(train_nn)
test_nn=transfer_device(test_nn)

In [26]:
# class CustomDataset(Dataset):
#     def __init__(self, data):
#         self.df = data[:,0:-1]
#         self.target = data[:, -1]

#     def __len__(self):
#         return len(self.target)

#     def __getitem__(self, idx):
#         target = self.target[idx]
#         data = self.df[idx]
#         return (data, target)

In [100]:
NUM_ITERATION = 10
NUM_MINIBATCH = 256

INPUT_L = 40
OUTPUT_L = 60

INPUT_H1 = 60
OUTPUT_H1 = 20

INPUT_H2 = 20
OUTPUT_H2 = 20

In [101]:
DEVICE= "cpu"

In [102]:
class ZaidNet(nn.Module):
    def __init__(self):
        super(ZaidNet, self).__init__()
        
        self.input_layer = nn.Linear(INPUT_L, OUTPUT_L)
        
        self.hidden_layer1 = nn.Linear(INPUT_H1, OUTPUT_H1)
        self.hidden_layer2 = nn.Linear(INPUT_H2, OUTPUT_H2)
        
        self.out_layer = nn.Linear(OUTPUT_H2, 1)
        
    def forward(self, x):
        x = F.relu(self.input_layer(x))
        
        x = F.relu(self.hidden_layer1(x))
        
        x = F.relu(self.hidden_layer2(x))
        
        x = self.out_layer(x)
        
        pred = torch.sigmoid(x)
        
        return pred

In [103]:
model = ZaidNet()
model = model.double()

In [104]:
LOSS = nn.BCELoss()
OPTIMIZER = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.00001)

In [105]:
model.to(DEVICE)

ZaidNet(
  (input_layer): Linear(in_features=40, out_features=60, bias=True)
  (hidden_layer1): Linear(in_features=60, out_features=20, bias=True)
  (hidden_layer2): Linear(in_features=20, out_features=20, bias=True)
  (out_layer): Linear(in_features=20, out_features=1, bias=True)
)

In [106]:
d_loader = DataLoader(train_nn, batch_size=256, shuffle=True)

In [107]:
for epoch in range(150):
    for i in range(len(train_nn)//256):
            data = next(iter(d_loader))
            train_x = data[:,0:-1]
            train_y = data[:,-1]



            y_hat = model(train_x).reshape((256,))

            cost = LOSS(y_hat, train_y)

            cost.backward()

            OPTIMIZER.step()
            OPTIMIZER.zero_grad()
    print(f'Epoch: {epoch+1}:',f"cost: {cost}")


KeyboardInterrupt: 

In [93]:
test = np.array(test)
test = torch.from_numpy(test)

In [94]:
X= train_nn[:, 0:-1]

In [95]:
sub_pred = model(test)

In [96]:
submission.target = sub_pred.detach().numpy()

In [97]:
submission.to_csv("nn_submission", index=False)

In [99]:
submission

,id,target
0,900000,0.481318
1,900001,0.481318
2,900002,0.481318
3,900003,0.481318
4,900004,0.481318
...,...,...
699995,1599995,0.481318
699996,1599996,0.481318
699997,1599997,0.481318
699998,1599998,0.481318
